In [13]:
!pip install torch torchvision

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/abaxter/.local/lib/python3.9/site-packages
sysconfig: /home/abaxter/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [ ]:
!pip install scikit-learn

!pip install pygments


: 

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import os
import pandas as pd
import torch.nn.functional as F


: 

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "gpu")
print(device)

cuda


In [4]:
class CSVDataset(Dataset):
    def __init__(self, dataframe, image_root_dir, target_columns=None, transform=None,
                 save_dir=None, use_saved_images=False):
        self.data = dataframe
        self.image_root_dir = image_root_dir
        self.target_columns = target_columns
        self.transform = transform
        self.save_dir = save_dir
        self.use_saved_images = use_saved_images

        if self.save_dir:
            os.makedirs(self.save_dir, exist_ok=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Use index for the saved tensor filename
        #saved_image_path = os.path.join(self.save_dir, f"{idx}.pt")
        image_index = row['Unnamed: 0']
        saved_image_path = os.path.join(self.save_dir, f"{image_index}.pt")



        if self.use_saved_images:
            if os.path.exists(saved_image_path):
                image_tensor = torch.load(saved_image_path)
            else:
                raise FileNotFoundError(f"Saved tensor not found: {saved_image_path}")
        else:
            original_image_path = os.path.join(self.image_root_dir, row['Path'])
            image = Image.open(original_image_path).convert("L")
            image_tensor = self.transform(image) if self.transform else transforms.ToTensor()(image)

            if self.save_dir:
                torch.save(image_tensor, saved_image_path)

        if self.target_columns:
            labels = pd.to_numeric(row[self.target_columns], errors='coerce').fillna(0).astype(float).values
            labels = torch.tensor(labels, dtype=torch.float32)
            return image_tensor, labels

        return image_tensor

In [5]:
class MultiLabelResNet50(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelResNet50, self).__init__()
        
        # Load pre-trained ResNet50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Modify the fully connected layer for multi-label classification
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 512),  # New intermediate layer
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout to prevent overfitting
            nn.Linear(512, num_classes),  # Output layer
            #nn.Sigmoid()  # Sigmoid for multi-label classification
            nn.Tanh()  #This is between -1 and 1

           # nn.Linear(self.base_model.fc.in_features, num_classes),
           # nn.Sigmoid()  # Sigmoid activation for multi-label classification
        )

    def forward(self, x):
        return self.base_model(x)

In [4]:
import torch.nn as nn
from torchvision import models

class MultiLabelDenseNet121(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelDenseNet121, self).__init__()
        
        # Load pretrained DenseNet-121
        self.base_model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        
        # Replace classifier (DenseNet has classifier instead of .fc)
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
            nn.Tanh()  # Output between -1 and 1 (matches your label format)
        )

    def forward(self, x):
        return self.base_model(x)


In [6]:
image_root = '/central/groups/CS156b/2025/CodeMonkeys/input_images'
image_root_dir = "train"
train_save_dir = os.path.join(image_root, 'train')
#test_save_dir = os.path.join(image_root, 'test')
full_train_df = pd.read_csv('train2023.csv')
#filtered_train_df = full_train_df.iloc[:29692]
filtered_train_df = full_train_df.iloc[:5000]



In [7]:
from sklearn.model_selection import train_test_split

# Define your target columns once
target_columns = [
    "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity",
    "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"
]

# Step 1: Split the dataframe
train_df, val_df = train_test_split(filtered_train_df, test_size=0.15, random_state=42)

# Step 2: Create training dataset
train_dataset = CSVDataset(
    dataframe=train_df, 
    image_root_dir=image_root, 
    target_columns=target_columns, 
    save_dir=train_save_dir, 
    use_saved_images=True
)

# Step 3: Create validation dataset
val_dataset = CSVDataset(
    dataframe=val_df, 
    image_root_dir=image_root, 
    target_columns=target_columns, 
    save_dir=train_save_dir, 
    use_saved_images=True
)


#Weighted Random Sampling: force each batch to have at least one known lavel

# Step 4: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [8]:
from torch.utils.data import WeightedRandomSampler

weights = []
for _, row in train_df.iterrows():
    labels = row[target_columns].fillna(torch.nan).values.astype(float)
    known = (labels != -1.0) & (~pd.isna(labels))
    weights.append(known.sum() + 1e-6)  # Avoid 0 weight

sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)


/tmp/ipykernel_2197222/2717735754.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels = row[target_columns].fillna(torch.nan).values.astype(float)
/tmp/ipykernel_2197222/2717735754.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels = row[target_columns].fillna(torch.nan).values.astype(float)
/tmp/ipykernel_2197222/2717735754.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `p

In [ ]:
def freeze_base_layers(model, until_layer=6):
    child_counter = 0
    for child in model.base_model.children():
        if child_counter < until_layer:
            for param in child.parameters():
                param.requires_grad = False
        child_counter += 1
    return model


In [10]:
def chexpert_loss(outputs, targets):
    valid_mask = ~torch.isnan(targets)
    soft_targets = targets.clone()
    uncertain_mask = (targets == -1.0)
    soft_targets[uncertain_mask] = 0.5
    weight = torch.ones_like(targets)
    weight[uncertain_mask] = 0.2
    weight[~valid_mask] = 0.0
    bce = F.binary_cross_entropy_with_logits(outputs, soft_targets, reduction='none')
    loss = bce * weight
    return loss.sum() / valid_mask.sum()

# import torch.nn.functional as F

# def chexpert_loss(outputs, targets):
#     # Map labels: -1 → 0.0, 0 → 0.5, 1 → 1.0
#     soft_targets = targets.clone()
#     soft_targets[targets == -1.0] = 0.0
#     soft_targets[targets ==  0.0] = 0.5
#     soft_targets[targets ==  1.0] = 1.0

#     # Apply BCEWithLogits loss
#     loss = F.binary_cross_entropy_with_logits(outputs, soft_targets, reduction='mean')
#     return loss



In [15]:

def masked_MSE_loss(output, target):
    mask = ~torch.isnan(target)
    loss = criterion(output, target)
    return loss[mask].mean()


'''#This one better apparently?
criterion = nn.BCEWithLogitsLoss(reduction='none')  # No reduction yet

def masked_BCE_loss(output, target):
    # Create mask for non-NaN targets
    mask = ~torch.isnan(target)  # Same shape as output/target

    # Compute unreduced BCE loss
    loss = criterion(output, target)

    # Apply mask and compute mean over valid entries
    masked_loss = loss[mask]
    return masked_loss.mean()'''


"#This one better apparently?\ncriterion = nn.BCEWithLogitsLoss(reduction='none')  # No reduction yet\n\ndef masked_BCE_loss(output, target):\n    # Create mask for non-NaN targets\n    mask = ~torch.isnan(target)  # Same shape as output/target\n\n    # Compute unreduced BCE loss\n    loss = criterion(output, target)\n\n    # Apply mask and compute mean over valid entries\n    masked_loss = loss[mask]\n    return masked_loss.mean()"

In [11]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np

def evaluate_model(model, val_loader):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = chexpert_loss(outputs, labels)
            running_loss += loss.item()

            # Convert logits to probabilities
            probs = torch.sigmoid(outputs)

            # Binarize predictions at 0.5 threshold
            preds = (probs > 0.5).float()

            # Only evaluate known labels (ignore NaN and -1.0)
            mask = (labels == 0.0) | (labels == 1.0)

            all_preds.append(preds[mask].cpu().numpy())
            all_labels.append(labels[mask].cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    # Compute metrics
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    accuracy = accuracy_score(all_labels, all_preds)

    print(f"Val Loss: {running_loss / len(val_loader):.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")
    print(f"Precision (Macro): {precision:.4f}")
    print(f"Recall (Macro): {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

    return running_loss / len(val_loader)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# List of CheXpert class names
class_names = [
    "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity",
    "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"
]

# Function to print per-class precision/recall/F1
def print_per_class_metrics(model, val_loader, class_names, device):
    model.eval()
    all_preds = [[] for _ in class_names]
    all_labels = [[] for _ in class_names]

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()

            for i in range(len(class_names)):
                mask = (labels[:, i] == 0.0) | (labels[:, i] == 1.0)
                all_preds[i].extend(preds[:, i][mask].cpu().numpy())
                all_labels[i].extend(labels[:, i][mask].cpu().numpy())

    print("\n📊 Per-Class Metrics:")
    for i, name in enumerate(class_names):
        try:
            p = precision_score(all_labels[i], all_preds[i], zero_division=0)
            r = recall_score(all_labels[i], all_preds[i], zero_division=0)
            f1 = f1_score(all_labels[i], all_preds[i], zero_division=0)
            print(f"{name:25s} | Precision: {p:.3f} | Recall: {r:.3f} | F1: {f1:.3f}")
        except:
            print(f"{name:25s} | No valid samples in this batch.")


In [ ]:
import torch.optim as optim
import numpy as np


num_classes = 9  # Adjust based on your number of labels
model = MultiLabelResNet50(num_classes=num_classes).to(device)
model = freeze_base_layers(model, until_layer=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Loss function and optimizer

optimizer = optim.Adam(model.parameters(), lr=0.001) #Could try 0.0001

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = chexpert_loss(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        val_loss = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}")
        # 🔻 Validation + Per-Class Stats
        val_loss = evaluate_model(model, val_loader)
        print_per_class_metrics(model, val_loader, class_names, device)

 



Val Loss: 0.4287
F1 Score (Macro): 0.6763
Precision (Macro): 0.7469
Recall (Macro): 0.6517
Accuracy: 0.8218
Epoch [1/10] Train Loss: 0.4422, Val Loss: 0.4287
Val Loss: 0.4309
F1 Score (Macro): 0.7557
Precision (Macro): 0.7439
Recall (Macro): 0.7718
Accuracy: 0.8265
Epoch [2/10] Train Loss: 0.4305, Val Loss: 0.4309
Val Loss: 0.4173
F1 Score (Macro): 0.7450
Precision (Macro): 0.7668
Recall (Macro): 0.7295
Accuracy: 0.8406
Epoch [3/10] Train Loss: 0.4272, Val Loss: 0.4173
Val Loss: 0.4199
F1 Score (Macro): 0.7204
Precision (Macro): 0.7667
Recall (Macro): 0.6964
Accuracy: 0.8359
Epoch [4/10] Train Loss: 0.4217, Val Loss: 0.4199
Val Loss: 0.4188
F1 Score (Macro): 0.7474
Precision (Macro): 0.7602
Recall (Macro): 0.7371
Accuracy: 0.8380
Epoch [5/10] Train Loss: 0.4199, Val Loss: 0.4188
Val Loss: 0.4199
F1 Score (Macro): 0.7508
Precision (Macro): 0.7581
Recall (Macro): 0.7444
Accuracy: 0.8373
Epoch [6/10] Train Loss: 0.4209, Val Loss: 0.4199
Val Loss: 0.4183
F1 Score (Macro): 0.7616
Precision 

KeyboardInterrupt: 

In [ ]:
from itertools import product

# --- Grid Search ---
param_grid = {
    "lr": [1e-4, 1e-3],
    "batch_size": [16, 32, 64],
    "freeze_until": [0, 3, 5]
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 9
num_epochs = 1
best_f1 = 0
best_params = None
results = []

for lr, batch_size, freeze_until in product(*param_grid.values()):
    print(f"Testing config: LR={lr}, BS={batch_size}, Freeze={freeze_until}")
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    model = MultiLabelDenseNet121(num_classes).to(device)
    model = freeze_base_layers(model, until_layer=freeze_until)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for _ in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            loss = chexpert_loss(model(images), labels)
            loss.backward()
            optimizer.step()

    val_f1 = evaluate_model(model, val_loader)
    print(f"→ Validation F1: {val_f1:.4f}")
    results.append((lr, batch_size, freeze_until, val_f1))

    if val_f1 > best_f1:
        best_f1 = val_f1
        best_params = (lr, batch_size, freeze_until)

    del model, optimizer
    torch.cuda.empty_cache()

print("\n--- Grid Search Results ---")
for r in results:
    print(f"LR={r[0]:.0e}, BS={r[1]}, Freeze={r[2]} → F1={r[3]:.4f}")
print(f"\nBest config: LR={best_params[0]}, BS={best_params[1]}, Freeze={best_params[2]} → F1={best_f1:.4f}")

Testing config: LR=0.0001, BS=16, Freeze=0
Val Loss: 0.4098
F1 Score (Macro): 0.7683
Precision (Macro): 0.7843
Recall (Macro): 0.7557
Accuracy: 0.8523
→ Validation F1: 0.4098
Testing config: LR=0.0001, BS=16, Freeze=3
Val Loss: 0.4099
F1 Score (Macro): 0.7660
Precision (Macro): 0.7865
Recall (Macro): 0.7508
Accuracy: 0.8526
→ Validation F1: 0.4099
Testing config: LR=0.0001, BS=16, Freeze=5
Val Loss: 0.4100
F1 Score (Macro): 0.7767
Precision (Macro): 0.7809
Recall (Macro): 0.7728
Accuracy: 0.8524
→ Validation F1: 0.4100
Testing config: LR=0.0001, BS=32, Freeze=0
Val Loss: 0.4079
F1 Score (Macro): 0.7680
Precision (Macro): 0.7849
Recall (Macro): 0.7549
Accuracy: 0.8524
→ Validation F1: 0.4079
Testing config: LR=0.0001, BS=32, Freeze=3
Val Loss: 0.4086
F1 Score (Macro): 0.7645
Precision (Macro): 0.7790
Recall (Macro): 0.7529
Accuracy: 0.8493
→ Validation F1: 0.4086
Testing config: LR=0.0001, BS=32, Freeze=5
Val Loss: 0.4089
F1 Score (Macro): 0.7680
Precision (Macro): 0.7764
Recall (Macro)

In [ ]:
# Create dataset
test_dataset = CSVDataset(
    dataframe=df_first10rows_test, 
    image_root_dir=image_root, 
    target_columns=None, 
    transform=image_transforms,  # Pass the transform
    save_dir=test_save_dir, 
    use_saved_images=False  # Set to True if you want to load tensors from CSV
)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Iterate through batches
for batch_idx, (images) in enumerate(test_loader):
    print(f"Batch {batch_idx + 1}")
    print("Images shape:", images.shape)

## Run Model

In [26]:
model = MultiLabelResNet50(num_classes=9).to(device)
model.load_state_dict(torch.load('models/model_epoch_10.pth'))
model.eval()  

MultiLabelResNet50(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [ ]:
test_dir = 'test_images'

# List to hold predictions
predictions = []

# Iterate over the test set
for filename in os.listdir(test_dir):
    if filename.endswith(".pt"):  # Only process .pt files
        # Load the image tensor (e.g., ID.pt)
        image_path = os.path.join(test_dir, filename)
        image_tensor = torch.load(image_path).to(device)
        
        # Make the prediction
        with torch.no_grad():
            output = model(image_tensor)  # Get model output (size: [1, num_classes])
        
        # Apply thresholding and collect the predictions
        output = output.squeeze().cpu().numpy()  # Remove batch dimension and move to CPU
        
        # Format the output with values between -1 and 1
        predictions.append([filename.split('.')[0]] + list(output))

# Convert predictions to a pandas DataFrame
columns = ["Id", "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", 
           "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"]
df_predictions = pd.DataFrame(predictions, columns=columns)

# Save the predictions to a CSV file
df_predictions.to_csv('test_predictions.csv', index=False)

print("Predictions saved to 'test_predictions.csv'")
